# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770281171569                   -0.53    8.0    154ms
  2   -2.771684223591       -2.85       -1.30    1.0   92.3ms
  3   -2.771714227340       -4.52       -2.63    1.0   94.6ms
  4   -2.771714690100       -6.33       -4.00    1.0    444ms
  5   -2.771714715166       -7.60       -4.61    2.0    111ms
  6   -2.771714715244      -10.11       -5.63    1.0    127ms
  7   -2.771714715249      -11.23       -5.89    2.0    142ms
  8   -2.771714715250      -13.38       -5.94    1.0   97.8ms
  9   -2.771714715250      -12.85       -7.66    1.0    102ms
 10   -2.771714715250   +  -14.88       -7.43    3.0    127ms
 11   -2.771714715250      -14.18       -7.79    1.0    102ms
 12   -2.771714715250   +  -14.75       -8.72    1.0    103ms


-0.00013457365326035483

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770394789973                   -0.53    8.0    154ms
  2   -2.771767826563       -2.86       -1.31    1.0   97.7ms
  3   -2.771801096951       -4.48       -2.60    1.0   96.2ms
  4   -2.771802069844       -6.01       -3.89    2.0    112ms
  5   -2.771802074048       -8.38       -4.24    2.0    507ms
  6   -2.771802074456       -9.39       -5.10    1.0    123ms
  7   -2.771802074476      -10.71       -5.75    2.0    146ms
  8   -2.771802074476      -12.42       -6.37    1.0    116ms
  9   -2.771802074476      -14.12       -7.39    1.0   99.3ms
 10   -2.771802074476      -14.35       -7.54    2.0    118ms
 11   -2.771802074476   +  -15.05       -7.97    1.0    105ms
 12   -2.771802074476      -14.24       -8.88    1.0    104ms


0.017612221652260587

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457365326035483
Displaced dipole:  0.017612221652260587
Polarizability :   1.7746795305520942


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, -δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   6.37s  Non-interacting
   1        0       1       -0.60     10.0   13.1s  
   2        0       2       -2.42      7.0   556ms  
   3        0       3       -3.55      5.0   102ms  
   4        0       4       -5.33      4.0  97.7ms  
   5        0       5       -7.59      1.0  85.2ms  
   6        0       6      -10.28      1.0  83.4ms  
   7        1       1       -7.41     13.0   670ms  Restart
   8        1       2       -8.91      1.0  82.5ms  
                                      13.0   720ms  Final orbitals


(δψ = Matrix{ComplexF64}[[0.0005037356848937484 - 0.003257172026770565im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.3053536683020215 - 0.04985310475536937im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; -0.04055706304025012 - 0.006439835495592833im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.08003408153035085 + 0.012063899173072538im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.620694070710789e-7 -2.502015223523877e-7 … -4.880640874041175e-8 -2.502014485424145e-7; -6.293616977714784e-7 -4.793036827875948e-7 … -1.2077842566426548e-7 -4.793036295309182e-7; … ; 1.3626263449376142e-7 1.1356231089320966e-7 … 4.7119915809636427e-8 1.1356232410499953e-7; 1.3305483068963825e-7 1.3632642539471823e-7 … 5.200687845470351e-8 1.3632648001943413e-7;;; -2.502014794662339e-7 -1.740490815559643e-7 … -3.5564925767791096e-8 -1.7404902783208868e-7; -4.793036526976034e-7 -3.6797322529984784e-7 … -1.1202893641050232e-7 -3.679731867254509e-7; … ; 1.1356231930574488e-7 1.121109966493008e-7 … 1.0980686204624324e-7 1.121

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125252732374
Interacting polarizability:     1.7736548560576695


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.29e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 5.22e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 5.83e-11
[ Info: GMRES linsolve in iteration 2; step 2: normres = 9.10e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 2.18e-13
└ * number of operations = 12
Non-interacting polarizability: 1.9257125252732041
Interacting polarizability:     1.7736548581189182


We obtain the identical result to above.